Say you have a bag of 99 fair coins and 1 coin that has heads on both sides.  You draw a coin from the bag and flip it 10 times, each time showing heads.  What is the probability you drew the unfair coin?

# Analytic Solution
We want to know the probability the coin drawn from the bag was the unfair coin, given it flipped heads 10 times.  

---
## Conditional Probability
To solve this analytically, first we need to learn about conditional probability.  Consider two events we will represent using $A$ and $B$.  For our example, the events will be:   

        A) drawing the unfair coin from the bag    
        B) flipping heads 10 times 

The probability of both events happening, $P(A\cap B)$, is equal to the probability of $A$ happening, $P(A)$, multiplied by the probability of $B$ happening given $A$ has occurred, $P(B|A)$.  

$$P(A\cap B) = P(B|A) P(A)$$

We could also switch this around, as $P(A\cap B)$ also is equal to the probability of $B$ happening, $P(B)$, multiplied by the probability of $A$ happening given $B$ has occurred, $P(A|B)$.

$$P(A\cap B) = P(A|B) P(B)$$

Putting these equalities together, we have 
$$P(A|B) P(B) = P(B|A) P(A)$$

which we can rewrite as
$$ P(A | B) = \frac{P(B | A) P(A)}{P(B)} $$

This is Bayes' theorem.

---
## What about the coin you drew?
To figure out the probability you drew the unfair coin, we have
- the probability of drawing the unfair coin from the bag: $P(A)$
- the probability of flipping heads 10 times: $P(B)$

We know the probability of drawing the unfair coin.
$$P(A) = 1 / 100$$

If we did draw the unfair coin, we know the probability of flipping heads 10 times will be

$$P(B | A) = 10 / 10 = 1$$

What about the probabilities related to drawing a fair coin, or not drawing the unfair coin, $P(!A)$?  The probability of not drawing the fair coin is $1 - P(A)$, or

$$P(!A) = 99 / 100$$

If we did draw a fair coin, we know the probability of flipping heads will be 1/2 for each flip, so for 10 flips we get

$$P(B | !A) = 1/2^{10}$$

From this we can calculate the probability of flipping nheads 10 times, $P(B)$, as

$$P(B) = P(B | A) P(A) + P(B | !A) P(!A)$$

or 

$$P(B) = 1 \times 1/100 + 1/2^{10} \times 99/100$$

and we have everything we need to solve the problem.

In [12]:
p_a = 1.0 / 100.0
p_ba = 1.0
p_na = 99.0 / 100.0
p_bna = 0.5 ** 10

p_b = p_ba * p_a + p_bna * p_na
p_ab = p_ba * p_a / p_b

print(f'Probability the drawn coin was unfair, given it flipped heads 10 times: {p_ab:0.3}')

Probability the drawn coin was unfair, given it flipped heads 10 times: 0.912


So the coin you drew is 91% likely to be the unfair coin.  **Is that as lower/higher than you thought?**

# Calculated Solution
Lets apply our Python hacking skills to instead simulate the experiment and see how often flipping 10 heads resulted from the unfair coin.

In [2]:
import numpy as np

Setup the coins:

In [3]:
# 100 coin all fair
n_coins = 100
coins = np.zeros(n_coins, dtype=np.int)  

# make one random coin unfair
coins[np.random.randint(1, n_coins, 1)] = 1

Define the coin draw-flip routine:

In [6]:
def draw_coin_and_flip(n_repeats, coins, *, n_flips=10, all_flips_heads=[]):
    n_coins = len(coins)
    
    for i in range(n_repeats):
        coin_index = np.random.randint(0, n_coins, 1)[0]
        drawn_coin = coins[coin_index]
        coin_unfair = drawn_coin == 1  # did we get the unfair coin?
        
        for j in range(n_flips):
            flip = np.random.rand()  # random number between [0, 1)
            
            if coin_unfair or flip < 0.5:
                heads = True
            else:
                heads = False
                break
        
        if heads:
            # store the index and if the coin was unfair
            all_flips_heads.append([i, coin_unfair])  
    
    return all_flips_heads

Simulate the coin draw-flip routine a million times.

In [7]:
n_repeats = int(1e6)
first_million = draw_coin_and_flip(n_repeats, coins, n_flips=10)

In [13]:
# evaluate the results
n_all_heads = len(first_million)
print(f'Of {n_repeats} repeated tests, {n_all_heads} showed heads 10 times')

all_heads_coins = np.c_[np.array(first_million), np.empty(n_all_heads)]
for i in range(n_all_heads):
    all_heads_coins[i, 2] = all_heads_coins[:i, 1].sum() * 1.0 / (i + 1)
print(f'probability coin was unfair: {all_heads_coins[-1, 2]:.3}')

Of 1000000 repeated tests, 10883 showed heads 10 times
probability coin was unfair: 0.911


**This is the same probability we got from the analytic solution.**  Lets see how this changed over the course of the simulation.

In [14]:
import bokeh.plotting
from bokeh.palettes import Category10_10 as palette
bokeh.plotting.output_notebook()

Loading BokehJS ...

In [23]:
p = bokeh.plotting.figure(width=400, height=400,
    y_axis_label='fraction of times 10 heads was from the unfair coin',
    x_axis_label='trials (e6)')

p.line(all_heads_coins[:, 0]/n_repeats, all_heads_coins[:, 2], 
       color=palette[0], legend='simulated result')
p.line(all_heads_coins[[0, -1], 0]/n_repeats, [p_ab, p_ab], 
       color=palette[1], legend='analytic result')

p.legend.location = 'bottom_right'
bokeh.plotting.show(p)

We see the simulated result equilibrated rather quickly to a value close to the analytic result.  We expect that running for a longer number of iterations would further improve the agreement.

Lets run for another million steps, then combine the results.

In [29]:
n_repeats = int(1e6)
two_million = draw_coin_and_flip(n_repeats, coins, n_flips=10, all_flips_heads=first_million)

In [36]:
# evaluate the results
n_all_heads = len(two_million)
print(f'Of {n_repeats*2} repeated tests, {n_all_heads} showed heads 10 times.')

all_heads_coins = np.c_[np.array(two_million), np.empty(n_all_heads)]
for i in range(n_all_heads):
    all_heads_coins[i, 2] = all_heads_coins[:i, 1].sum() * 1.0 / (i + 1)
print(f'Of those {n_all_heads} all heads runs, {all_heads_coins[-1, 2]*100:0.3}% were the unfair coin') 

Of 2000000 repeated tests, 32606 showed heads 10 times.
Of those 32606 all heads runs, 90.9% were the unfair coin


This matches the analytic result to the precision shown.

# Fix these plots!

In [ ]:


p = bokeh.plotting.figure(width=400, height=400,
    y_axis_label='fraction of times 10 heads was from the unfair coin',
    x_axis_label='trials')

p.line(two_million[:, 0], both_heads_coins[:, 2], 
       color=palette[0], legend='simulated result')
p.line(both_heads_coins[[0, -1], 0], [p_ab, p_ab], 
       color=palette[1], legend='analytic result')

p.legend.location = 'bottom_right'
bokeh.plotting.show(p)

In [39]:
p = bokeh.plotting.figure(width=400, height=400,
    y_axis_label='fraction of times 10 heads was from the unfair coin',
    x_axis_label='trials (e6)')

p.line(all_heads_coins[:, 0]/(n_repeats/2), all_heads_coins[:, 2], 
       color=palette[0], legend='simulated result')
p.line(all_heads_coins[[0, -1], 0]/(n_repeats/2), [p_ab, p_ab], 
       color=palette[1], legend='analytic result')

p.legend.location = 'bottom_right'
bokeh.plotting.show(p)